In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

# Data cleaning

## Metabolites

Loading the raw data.

In [2]:
raw_data = pd.read_csv('../data/train_raw.csv').T

In [3]:
raw_data.head(10)

,0,1,2,3,4,5,6,7,8,9,...,44826,44827,44828,44829,44830,44831,44832,44833,44834,44835
row ID,36430.000000,19616.000000,14012.000000,29424.000000,40633.000000,36431.000000,29425.000000,14013.000000,19617.000000,12611.000000,...,9798.000000,9799.000000,9800.000000,9801.000000,9802.000000,9803.000000,9804.000000,9805.000000,9806.000000,9807.000000
row m/z,379.166221,219.065232,538.228789,134.043938,348.155837,379.167956,141.126867,538.326901,221.138249,159.138059,...,374.289242,374.289710,378.228959,379.234334,382.200928,393.297166,394.300548,398.312555,398.814153,406.268451
row retention time,5.558218,5.082561,4.364786,7.143080,2.270031,6.076645,6.784533,0.126516,0.025611,0.489101,...,0.372133,0.395216,5.149145,5.764760,7.103840,0.415677,0.453511,0.396557,0.402368,6.203453
row identity (main ID),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
row identity (all IDs),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
row identity (main ID + details),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
row comment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
row number of detected peaks,1.000000,6.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,4.000000,2.000000,...,1.000000,1.000000,2.000000,19.000000,23.000000,9.000000,6.000000,1.000000,1.000000,36.000000
20170904_NT_186_GA1_01_19777.mzXML Peak height,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,79.000000,170.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,561.000000,182.000000,25.000000,0.000000,0.000000
20170904_NT_186_GA1_01_19777.mzXML Peak area,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,61.434500,273.320000,...,0.000000,0.000000,0.000000,0.000000,0.000000,2892.080500,639.055500,7.694500,0.000000,0.000000


The first we do is identify the metabolites. For cleanliness sake, we create an auxiliary table linking the column number to the metabolite properties. In other words, we will work with metabolites 0, 1, 2, etc., while the auxiliary table will tell us that, *eg.*, metabolite 0 has an m/z ratio of 379.17 and a retention time of 5.56 min.

In [4]:
peak_IDs = raw_data.T[['row m/z', 'row retention time', 'row number of detected peaks']].rename(index=str,
                       columns={'row m/z': 'm/z', 'row retention time': 'retention time',
                                'row number of detected peaks': 'n peaks'})
peak_IDs.head()

,m/z,retention time,n peaks
0,379.166221,5.558218,1.0
1,219.065232,5.082561,6.0
2,538.228789,4.364786,1.0
3,134.043938,7.143080,1.0
4,348.155837,2.270031,1.0


In [ ]:
peak_IDs.to_csv('../data/peak_IDs.csv', index=False)

With that out of the way, we remove the rows with the metabolite IDs from the dataset, along with the rest of the rows containing unnecessary data, so that we are left with rows of samples and columns of metabolites.

In [5]:
data_all = raw_data.drop(raw_data.index[[i for i in range(8)]])
data_all.head()

,0,1,2,3,4,5,6,7,8,9,...,44826,44827,44828,44829,44830,44831,44832,44833,44834,44835
20170904_NT_186_GA1_01_19777.mzXML Peak height,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79.0000,170.000,...,0.0,0.0000,0.0,0.000,0.000,561.0000,182.0000,25.0000,0.0,0.000
20170904_NT_186_GA1_01_19777.mzXML Peak area,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61.4345,273.320,...,0.0,0.0000,0.0,0.000,0.000,2892.0805,639.0555,7.6945,0.0,0.000
20170904_NT_193_GA8_01_19784.mzXML Peak height,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,36.000,...,0.0,25.0000,0.0,623.000,1039.000,2395.0000,628.0000,0.0000,0.0,499.000
20170904_NT_193_GA8_01_19784.mzXML Peak area,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,11.189,...,0.0,5.8625,0.0,3077.788,5427.048,12962.4355,2937.6970,0.0000,0.0,3853.611
20170904_NT_191_GA6_01_19782.mzXML Peak height,0.0,0.0,0.0,0.0,645.0,0.0,33.0,0.0,58.0000,33.000,...,0.0,0.0000,0.0,690.000,1257.000,2334.0000,689.0000,0.0000,0.0,877.000


We have two datasets in one: peak AUC and peak intensity for each one of the samples. Before separating them, we note that:
1. We can clean up the sample ID by leaving only the last five digits.
2. We need to take care of the blanks, marked by the flag **ACN**. They can be found towards the end of the dataset.
3. The very last row of the data contains no information.

Therefore, we do the following:
1. Remove the last row.
2. Create an additional boolean column stating whether a sample is a blank or not.
3. Remove unnecessary information from the sample ID and sort them by number.

In [6]:
data_all = data_all.drop(data_all.index[-1])
data_all.tail()

,0,1,2,3,4,5,6,7,8,9,...,44826,44827,44828,44829,44830,44831,44832,44833,44834,44835
20170904_NT_557_RD6_01_20191.mzXML Peak area,0.0,9291.3975,0.0,0.0,322.8525,0.0,0.0,0.0,0.0,636.437,...,0.0,5.344,0.0,99928.3350,0.0,59889.896,9189.4915,0.0000,59.285,50263.362
20170904_NT_556_RD5_01_20190.mzXML Peak height,0.0,558.0000,0.0,0.0,250.0000,0.0,0.0,0.0,0.0,246.000,...,0.0,29.000,0.0,14442.0000,0.0,9752.000,1878.0000,1849.0000,474.000,5602.000
20170904_NT_556_RD5_01_20190.mzXML Peak area,0.0,1829.4650,0.0,0.0,247.1180,0.0,0.0,0.0,0.0,372.410,...,0.0,8.517,0.0,88927.0035,0.0,34783.094,4291.7330,2116.0045,658.481,32364.270
20170904_NT_550_RC7_01_20184.mzXML Peak height,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,79.000,...,0.0,0.000,0.0,0.0000,0.0,553.000,202.0000,0.0000,0.000,0.000
20170904_NT_550_RC7_01_20184.mzXML Peak area,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,75.818,...,0.0,0.000,0.0,0.0000,0.0,2604.031,606.0430,0.0000,0.000,0.000


In [7]:
blanks = ['ACN' in x for x in data_all.index]
data_all['blank'] = blanks
data_all.head()

,0,1,2,3,4,5,6,7,8,9,...,44827,44828,44829,44830,44831,44832,44833,44834,44835,blank
20170904_NT_186_GA1_01_19777.mzXML Peak height,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79.0000,170.000,...,0.0000,0.0,0.000,0.000,561.0000,182.0000,25.0000,0.0,0.000,False
20170904_NT_186_GA1_01_19777.mzXML Peak area,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61.4345,273.320,...,0.0000,0.0,0.000,0.000,2892.0805,639.0555,7.6945,0.0,0.000,False
20170904_NT_193_GA8_01_19784.mzXML Peak height,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,36.000,...,25.0000,0.0,623.000,1039.000,2395.0000,628.0000,0.0000,0.0,499.000,False
20170904_NT_193_GA8_01_19784.mzXML Peak area,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,11.189,...,5.8625,0.0,3077.788,5427.048,12962.4355,2937.6970,0.0000,0.0,3853.611,False
20170904_NT_191_GA6_01_19782.mzXML Peak height,0.0,0.0,0.0,0.0,645.0,0.0,33.0,0.0,58.0000,33.000,...,0.0000,0.0,690.000,1257.000,2334.0000,689.0000,0.0000,0.0,877.000,False


In [8]:
y = [x[-x[::-1].find('_'):] for x in data_all.index]
data_all.index = y
data_all.head()

,0,1,2,3,4,5,6,7,8,9,...,44827,44828,44829,44830,44831,44832,44833,44834,44835,blank
19777.mzXML Peak height,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79.0000,170.000,...,0.0000,0.0,0.000,0.000,561.0000,182.0000,25.0000,0.0,0.000,False
19777.mzXML Peak area,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61.4345,273.320,...,0.0000,0.0,0.000,0.000,2892.0805,639.0555,7.6945,0.0,0.000,False
19784.mzXML Peak height,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,36.000,...,25.0000,0.0,623.000,1039.000,2395.0000,628.0000,0.0000,0.0,499.000,False
19784.mzXML Peak area,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,11.189,...,5.8625,0.0,3077.788,5427.048,12962.4355,2937.6970,0.0000,0.0,3853.611,False
19782.mzXML Peak height,0.0,0.0,0.0,0.0,645.0,0.0,33.0,0.0,58.0000,33.000,...,0.0000,0.0,690.000,1257.000,2334.0000,689.0000,0.0000,0.0,877.000,False


In [9]:
data_all = data_all.sort_index()
data_all.head()

,0,1,2,3,4,5,6,7,8,9,...,44827,44828,44829,44830,44831,44832,44833,44834,44835,blank
19775.mzXML Peak area,15.732,42.938,4.446,23.7835,9.063,0.000,50.7870,59.285,4614.9215,326.652,...,0.0,5.643,40.030,0.000,5879.2350,1554.7700,12.1910,14.362,36.6255,True
19775.mzXML Peak height,38.000,42.000,14.000,47.0000,31.000,0.000,55.0000,189.000,3716.0000,355.000,...,0.0,23.000,39.000,0.000,2945.0000,722.0000,21.0000,54.000,46.0000,True
19776.mzXML Peak area,0.000,6.327,0.000,28.2340,0.000,53.198,40.5625,0.000,109.0510,527.656,...,0.0,0.000,37.962,7.011,5025.5405,1307.7660,11.8570,0.000,10.2660,True
19776.mzXML Peak height,0.000,24.000,0.000,39.0000,0.000,46.000,49.0000,0.000,154.0000,402.000,...,0.0,0.000,50.000,22.000,4502.0000,1109.0000,21.0000,0.000,12.0000,True
19777.mzXML Peak area,0.000,0.000,0.000,0.0000,0.000,0.000,0.0000,0.000,61.4345,273.320,...,0.0,0.000,0.000,0.000,2892.0805,639.0555,7.6945,0.000,0.0000,False


We ignore media blanks.

In [11]:
to_drop = ['20080', '20093', '20107', '20123', '20140', '20141', '20167', '20168']

height = '.mzXML Peak height'
area = '.mzXML Peak area'

for x in to_drop:
    if x+height in data_all.index:
        data_all = data_all.drop([x+area, x+height])

In [13]:
data_all.tail()

,0,1,2,3,4,5,6,7,8,9,...,44827,44828,44829,44830,44831,44832,44833,44834,44835,blank
20192.mzXML Peak height,0.0000,0.000,0.0,0.0000,0.0,0.000,0.0000,0.0,0.0,422.000,...,0.000,0.000,0.000,0.0000,15640.000,3939.000,0.000,110.000,0.000,False
20193.mzXML Peak area,15.5700,33.173,0.0,0.0000,0.0,58.766,20.6910,0.0,0.0,1941.709,...,54.609,0.000,25.137,4.9590,47809.595,11183.322,3203.121,617.065,11.286,True
20193.mzXML Peak height,26.0000,35.000,0.0,0.0000,0.0,42.000,24.0000,0.0,0.0,823.000,...,170.000,0.000,30.000,21.0000,9729.000,2833.000,1194.000,552.000,36.000,True
20195.mzXML Peak area,39.3745,3.933,0.0,50.9945,0.0,11.286,23.2645,0.0,0.0,48.263,...,0.000,9.234,0.000,25.1865,1656.807,380.426,3.674,0.000,0.000,True
20195.mzXML Peak height,38.0000,14.000,0.0,50.0000,0.0,33.000,32.0000,0.0,0.0,43.000,...,0.000,35.000,0.000,31.0000,297.000,174.000,11.000,0.000,0.000,True


Now, it is time to separate the intensity and AUC datasets, and keep only the sample ID.

In [17]:
intensity = ['height' in x for x in data_all.index]
area = ['area' in x for x in data_all.index]

data_int = data_all[intensity]
data_auc = data_all[area]

new_index = [x[:x.find('.')] for x in data_int.index]
data_int.index = new_index
data_auc.index = new_index

In [28]:
data_int.head()

,0,1,2,3,4,5,6,7,8,9,...,44827,44828,44829,44830,44831,44832,44833,44834,44835,blank
19775,38.0,42.0,14.0,47.0,31.0,0.0,55.0,189.0,3716.0,355.0,...,0.0,23.0,39.0,0.0,2945.0,722.0,21.0,54.0,46.0,True
19776,0.0,24.0,0.0,39.0,0.0,46.0,49.0,0.0,154.0,402.0,...,0.0,0.0,50.0,22.0,4502.0,1109.0,21.0,0.0,12.0,True
19777,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79.0,170.0,...,0.0,0.0,0.0,0.0,561.0,182.0,25.0,0.0,0.0,False
19778,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,86.0,...,907.0,0.0,0.0,3806.0,720.0,272.0,122.0,81.0,0.0,False
19779,0.0,0.0,0.0,0.0,622.0,0.0,0.0,0.0,0.0,124.0,...,92.0,2590.0,2399.0,2348.0,1889.0,643.0,85.0,36.0,1797.0,False


In [25]:
data_auc.head()

,0,1,2,3,4,5,6,7,8,9,...,44827,44828,44829,44830,44831,44832,44833,44834,44835,blank
19775,15.732,42.938,4.446,23.7835,9.063,0.000,50.7870,59.285,4614.9215,326.652,...,0.000,5.6430,40.030,0.0000,5879.2350,1554.7700,12.1910,14.362,36.6255,True
19776,0.000,6.327,0.000,28.2340,0.000,53.198,40.5625,0.000,109.0510,527.656,...,0.000,0.0000,37.962,7.0110,5025.5405,1307.7660,11.8570,0.000,10.2660,True
19777,0.000,0.000,0.000,0.0000,0.000,0.000,0.0000,0.000,61.4345,273.320,...,0.000,0.0000,0.000,0.0000,2892.0805,639.0555,7.6945,0.000,0.0000,False
19778,0.000,0.000,0.000,0.0000,0.000,0.000,0.0000,0.000,30.7280,98.530,...,765.194,0.0000,0.000,10583.8075,3443.8295,987.9465,105.0430,64.629,0.0000,False
19779,0.000,0.000,0.000,0.0000,2038.306,0.000,0.0000,0.000,0.0000,246.914,...,36.072,8490.1125,9301.011,6930.0320,10912.5705,2433.1070,53.7675,17.709,10839.2135,False


We need to subtract the each blank from the batch of samples that follows it. We start by removing all but the last blank whenever we encounter consecutive blanks. We also remove the very last rows of the data, since no samples follow these blanks.

In [49]:
# locations of blanks followed by another blank
idx_consec = [i for i in range(data_int.shape[0]-1) if data_int.iloc[i].blank and data_int.iloc[i+1].blank]

# remove these rows
data_int = data_int.drop(data_int.index[idx_consec])
data_auc = data_auc.drop(data_auc.index[idx_consec])

# remove last row
data_int = data_int.drop(data_int.index[-1])
data_auc = data_auc.drop(data_auc.index[-1])

Now, we add another column indicating the batch number. For each batch, we subtract the first row from all the other rows.

In [57]:
# batch number
batches = np.cumsum(data_auc.blank)

data_int['batch'] = batches
data_auc['batch'] = batches

In [59]:
data_int.head(12)

,0,1,2,3,4,5,6,7,8,9,...,44828,44829,44830,44831,44832,44833,44834,44835,blank,batch
19776,0.0,24.0,0.0,39.0,0.0,46.0,49.0,0.0,154.0,402.0,...,0.0,50.0,22.0,4502.0,1109.0,21.0,0.0,12.0,True,1
19777,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79.0,170.0,...,0.0,0.0,0.0,561.0,182.0,25.0,0.0,0.0,False,1
19778,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,86.0,...,0.0,0.0,3806.0,720.0,272.0,122.0,81.0,0.0,False,1
19779,0.0,0.0,0.0,0.0,622.0,0.0,0.0,0.0,0.0,124.0,...,2590.0,2399.0,2348.0,1889.0,643.0,85.0,36.0,1797.0,False,1
19780,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.0,31.0,...,0.0,3646.0,0.0,1889.0,605.0,30.0,35.0,2030.0,False,1
19781,0.0,0.0,0.0,72.0,775.0,0.0,28.0,0.0,40.0,37.0,...,0.0,0.0,68.0,2474.0,642.0,22.0,0.0,0.0,False,1
19782,0.0,0.0,0.0,0.0,645.0,0.0,33.0,0.0,58.0,33.0,...,0.0,690.0,1257.0,2334.0,689.0,0.0,0.0,877.0,False,1
19783,0.0,0.0,0.0,0.0,478.0,0.0,0.0,0.0,0.0,33.0,...,0.0,2613.0,0.0,2725.0,641.0,22.0,0.0,891.0,False,1
19784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,...,0.0,623.0,1039.0,2395.0,628.0,0.0,0.0,499.0,False,1
19785,0.0,0.0,19.0,17.0,19.0,39.0,0.0,0.0,0.0,18.0,...,0.0,29.0,20.0,3187.0,824.0,0.0,0.0,24.0,True,2


In [97]:
for batch in np.unique(batches):
    idx_batch = data_int[data_int.batch == batch].index.values
    
    blank_mask = data_int.loc[idx_batch].blank
    
    x_int = data_int.loc[idx_batch].drop(['blank', 'batch'],axis=1)-data_int.drop(['blank', 'batch'],axis=1).loc[idx_batch[0]]
    x_auc = data_auc.loc[idx_batch].drop(['blank', 'batch'],axis=1)-data_auc.drop(['blank', 'batch'],axis=1).loc[idx_batch[0]]
    
    x_int['blank'] = blank_mask
    x_int['batch'] = batch
    
    x_auc['blank'] = blank_mask
    x_auc['batch'] = batch
    
    data_int.loc[idx_batch] = x_int
    data_auc.loc[idx_batch] = x_auc

In [98]:
data_int.head(12)

,0,1,2,3,4,5,6,7,8,9,...,44828,44829,44830,44831,44832,44833,44834,44835,blank,batch
19776,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,1
19777,0.0,-24.0,0.0,-39.0,0.0,-46.0,-49.0,0.0,-75.0,-232.0,...,0.0,-50.0,-22.0,-3941.0,-927.0,4.0,0.0,-12.0,False,1
19778,0.0,-24.0,0.0,-39.0,0.0,-46.0,-49.0,0.0,-104.0,-316.0,...,0.0,-50.0,3784.0,-3782.0,-837.0,101.0,81.0,-12.0,False,1
19779,0.0,-24.0,0.0,-39.0,622.0,-46.0,-49.0,0.0,-154.0,-278.0,...,2590.0,2349.0,2326.0,-2613.0,-466.0,64.0,36.0,1785.0,False,1
19780,0.0,-24.0,0.0,-39.0,0.0,-46.0,-49.0,0.0,-109.0,-371.0,...,0.0,3596.0,-22.0,-2613.0,-504.0,9.0,35.0,2018.0,False,1
19781,0.0,-24.0,0.0,33.0,775.0,-46.0,-21.0,0.0,-114.0,-365.0,...,0.0,-50.0,46.0,-2028.0,-467.0,1.0,0.0,-12.0,False,1
19782,0.0,-24.0,0.0,-39.0,645.0,-46.0,-16.0,0.0,-96.0,-369.0,...,0.0,640.0,1235.0,-2168.0,-420.0,-21.0,0.0,865.0,False,1
19783,0.0,-24.0,0.0,-39.0,478.0,-46.0,-49.0,0.0,-154.0,-369.0,...,0.0,2563.0,-22.0,-1777.0,-468.0,1.0,0.0,879.0,False,1
19784,0.0,-24.0,0.0,-39.0,0.0,-46.0,-49.0,0.0,-154.0,-366.0,...,0.0,573.0,1017.0,-2107.0,-481.0,-21.0,0.0,487.0,False,1
19785,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,2


Now, we remove the blank rows, plus the blank and batch columns.

In [99]:
blank_rows = data_int[data_int.blank].index.values
blank_rows

array(['19776', '19785', '19794', '19803', '19813', '19823', '19833',
       '19843', '19853', '19863', '19872', '19882', '19892', '19902',
       '19912', '19922', '19932', '19939', '19949', '19959', '19969',
       '19979', '19989', '19999', '20009', '20019', '20029', '20039',
       '20049', '20059', '20069', '20072', '20083', '20094', '20105',
       '20116', '20127', '20138', '20149', '20160', '20171', '20182'],
      dtype=object)

In [104]:
data_int = data_int.drop(blank_rows).drop(['blank', 'batch'], axis=1)
data_auc = data_auc.drop(blank_rows).drop(['blank', 'batch'], axis=1)

In [123]:
print('Total number of samples: %i' % data_int.shape[0])

Total number of samples: 364


Now we have only samples. However, many of the entries are negative. We replace all of these with zeros.

In [117]:
data_int = data_int.clip_lower(0)
data_auc = data_auc.clip_lower(0)

In [118]:
data_int.head()

,0,1,2,3,4,5,6,7,8,9,...,44826,44827,44828,44829,44830,44831,44832,44833,44834,44835
19777,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
19778,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,933.0,907.0,0.0,0.0,3784.0,0.0,0.0,101.0,81.0,0.0
19779,0.0,0.0,0.0,0.0,622.0,0.0,0.0,0.0,0.0,0.0,...,0.0,92.0,2590.0,2349.0,2326.0,0.0,0.0,64.0,36.0,1785.0
19780,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3596.0,0.0,0.0,0.0,9.0,35.0,2018.0
19781,0.0,0.0,0.0,33.0,775.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,46.0,0.0,0.0,1.0,0.0,0.0


In [119]:
data_auc.head()

,0,1,2,3,4,5,6,7,8,9,...,44826,44827,44828,44829,44830,44831,44832,44833,44834,44835
19777,0.0,0.0,0.0,0.000,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.0000,0.000,0.0000,0.0000,0.000,0.0000,0.000,0.0000
19778,0.0,0.0,0.0,0.000,0.0000,0.0,0.0,0.0,0.0,0.0,...,304.608,765.194,0.0000,0.000,10576.7965,0.0000,0.000,93.1860,64.629,0.0000
19779,0.0,0.0,0.0,0.000,2038.3060,0.0,0.0,0.0,0.0,0.0,...,0.000,36.072,8490.1125,9263.049,6923.0210,5887.0300,1125.341,41.9105,17.709,10828.9475
19780,0.0,0.0,0.0,0.000,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.0000,10885.824,0.0000,3676.3155,737.354,0.0000,7.682,6254.5390
19781,0.0,0.0,0.0,143.821,1625.1515,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.0000,0.000,69.9490,7831.6225,1413.332,2.8390,0.000,0.0000


We create an additional dataset for the AUC, where we set to zero all peak areas for which the intensity was set to zero.

In [77]:
data_zauc = np.heaviside(data_int,0)*data_auc

data_zauc.head()

,0,1,2,3,4,5,6,7,8,9,...,44826,44827,44828,44829,44830,44831,44832,44833,44834,44835
19777,0.0,0.0,0.0,0.000,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.0000,0.000,0.0000,0.0,0.0,0.0000,0.000,0.0000
19778,0.0,0.0,0.0,0.000,0.0000,0.0,0.0,0.0,0.0,0.0,...,304.608,765.194,0.0000,0.000,10576.7965,0.0,0.0,93.1860,64.629,0.0000
19779,0.0,0.0,0.0,0.000,2038.3060,0.0,0.0,0.0,0.0,0.0,...,0.000,36.072,8490.1125,9263.049,6923.0210,0.0,0.0,41.9105,17.709,10828.9475
19780,0.0,0.0,0.0,0.000,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.0000,10885.824,0.0000,0.0,0.0,0.0000,7.682,6254.5390
19781,0.0,0.0,0.0,143.821,1625.1515,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.0000,0.000,69.9490,0.0,0.0,2.8390,0.000,0.0000


We save the resulting datasets.

In [78]:
data_int.to_csv('../data/data_int.csv')
data_auc.to_csv('../data/data_auc.csv')
data_zauc.to_csv('../data/data_zauc.csv')

## Metadata

Now that we have clean metabolite data, we can add the metadata to the samples. We note that there is metadata for only 228 of them.

In [20]:
# the data we want is in the 3rd sheet of the Excel file

metadata = pd.read_excel('../data/Metadata.xlsx', 3)
metadata.head()

,filename,ATTRIBUTE_SampleType,ATTRIBUTE_strain,ATTRIBUTE_genus,ATTRIBUTE_species,ATTRIBUTE_elevation,ATTRIBUTE_Location,ATTRIBUTE_pH,ATTRIBUTE_Temperature,ATTRIBUTE_Moisture,ATTRIBUTE_SoilType,ATTRIBUTE_media
0,20170904_NT_186_GA1_01_19777.mzXML,SAMPLE,409,Xenorhabdus,stockiae,186,Kaengchetkhaew_National_Park_Phitsanulok_Province,6.8,28,1,Loam,LB
1,20170904_NT_197_GB4_01_19789.mzXML,SAMPLE,409,Xenorhabdus,stockiae,186,Kaengchetkhaew_National_Park_Phitsanulok_Province,6.8,28,1,Loam,SF900
2,20170904_NT_222_GE5_01_19817.mzXML,SAMPLE,432,Photorhabdus,luminescens,243,Nakhonthai_District_Phitsanulok_Province,6.6,26,1.5,Sandy_Loam,LB
3,20170904_NT_223_GE6_01_19818.mzXML,SAMPLE,433,Photorhabdus,luminescens,243,Nakhonthai_District_Phitsanulok_Province,6.8,27,1,Sandy_Loam,LB
4,20170904_NT_224_GE7_01_19819.mzXML,SAMPLE,434,Photorhabdus,luminescens,255,Nakhonthai_District_Phitsanulok_Province,6.8,27,1,Sandy_Loam,LB


We don't need the blanks anymore, and we need to clean the sample ID.

In [21]:
# choosing only samples and removing the sample type since we don't need it anymore
metadata = metadata[metadata.ATTRIBUTE_SampleType == 'SAMPLE'].drop(['ATTRIBUTE_SampleType'], axis=1)
metadata.head()

,filename,ATTRIBUTE_strain,ATTRIBUTE_genus,ATTRIBUTE_species,ATTRIBUTE_elevation,ATTRIBUTE_Location,ATTRIBUTE_pH,ATTRIBUTE_Temperature,ATTRIBUTE_Moisture,ATTRIBUTE_SoilType,ATTRIBUTE_media
0,20170904_NT_186_GA1_01_19777.mzXML,409,Xenorhabdus,stockiae,186,Kaengchetkhaew_National_Park_Phitsanulok_Province,6.8,28,1,Loam,LB
1,20170904_NT_197_GB4_01_19789.mzXML,409,Xenorhabdus,stockiae,186,Kaengchetkhaew_National_Park_Phitsanulok_Province,6.8,28,1,Loam,SF900
2,20170904_NT_222_GE5_01_19817.mzXML,432,Photorhabdus,luminescens,243,Nakhonthai_District_Phitsanulok_Province,6.6,26,1.5,Sandy_Loam,LB
3,20170904_NT_223_GE6_01_19818.mzXML,433,Photorhabdus,luminescens,243,Nakhonthai_District_Phitsanulok_Province,6.8,27,1,Sandy_Loam,LB
4,20170904_NT_224_GE7_01_19819.mzXML,434,Photorhabdus,luminescens,255,Nakhonthai_District_Phitsanulok_Province,6.8,27,1,Sandy_Loam,LB


In [22]:
idmeta = metadata.filename.apply(lambda x: x[-x[::-1].find('_'):].strip('.mzXML')).rename()
metadata.index = idmeta
metadata = metadata.drop(['filename'], axis=1)
metadata.head()

,ATTRIBUTE_strain,ATTRIBUTE_genus,ATTRIBUTE_species,ATTRIBUTE_elevation,ATTRIBUTE_Location,ATTRIBUTE_pH,ATTRIBUTE_Temperature,ATTRIBUTE_Moisture,ATTRIBUTE_SoilType,ATTRIBUTE_media
19777,409,Xenorhabdus,stockiae,186,Kaengchetkhaew_National_Park_Phitsanulok_Province,6.8,28,1,Loam,LB
19789,409,Xenorhabdus,stockiae,186,Kaengchetkhaew_National_Park_Phitsanulok_Province,6.8,28,1,Loam,SF900
19817,432,Photorhabdus,luminescens,243,Nakhonthai_District_Phitsanulok_Province,6.6,26,1.5,Sandy_Loam,LB
19818,433,Photorhabdus,luminescens,243,Nakhonthai_District_Phitsanulok_Province,6.8,27,1,Sandy_Loam,LB
19819,434,Photorhabdus,luminescens,255,Nakhonthai_District_Phitsanulok_Province,6.8,27,1,Sandy_Loam,LB


Let's clean the column names

In [24]:
columns = {col: col.lower().replace('attribute_', '') for col in metadata.columns}
metadata = metadata.rename(columns=columns)
metadata.head()

,strain,genus,species,elevation,location,ph,temperature,moisture,soiltype,media
19777,409,Xenorhabdus,stockiae,186,Kaengchetkhaew_National_Park_Phitsanulok_Province,6.8,28,1,Loam,LB
19789,409,Xenorhabdus,stockiae,186,Kaengchetkhaew_National_Park_Phitsanulok_Province,6.8,28,1,Loam,SF900
19817,432,Photorhabdus,luminescens,243,Nakhonthai_District_Phitsanulok_Province,6.6,26,1.5,Sandy_Loam,LB
19818,433,Photorhabdus,luminescens,243,Nakhonthai_District_Phitsanulok_Province,6.8,27,1,Sandy_Loam,LB
19819,434,Photorhabdus,luminescens,255,Nakhonthai_District_Phitsanulok_Province,6.8,27,1,Sandy_Loam,LB


In [30]:
metadata.tail()

,strain,genus,species,elevation,location,ph,temperature,moisture,soiltype,media
20187,581,Xenorhabdus,stockiae,43,Bangkrathum_Sub-district_Bangkrathum_District_...,7,29,1,Loam,SF900
20188,583,Photorhabdus,luminescens,44,Prompiram_Sub-district_Prompiram_District_Phit...,7,29,1,Sand,SF900
20189,584,Xenorhabdus,stockiae,263,Takhian_Sub-district_Dankhunthot_District_Nakh...,7,31,1,Loam,SF900
20190,585,Photorhabdus,luminescens,253,Takhian_Sub-district_Dankhunthot_District_Nakh...,6.8,29,1,Sandy_Loam,SF900
20191,586,Photorhabdus,luminescens,1270,Pangsilathong_District_Kamphaengphet_province,6.2,18,2,Loam,SF900


Now we keep the location to the province level

In [42]:
metadata.location = metadata.location.apply(lambda x: x.lower().replace('_province', '')[-x.lower().replace('_province', '')[::-1].find('_'):])
metadata.rename(columns={'location': 'province'}, inplace=True)
metadata.head()

,strain,genus,species,elevation,province,ph,temperature,moisture,soiltype,media
19777,409,Xenorhabdus,stockiae,186,phitsanulok,6.8,28,1,Loam,LB
19789,409,Xenorhabdus,stockiae,186,phitsanulok,6.8,28,1,Loam,SF900
19817,432,Photorhabdus,luminescens,243,phitsanulok,6.6,26,1.5,Sandy_Loam,LB
19818,433,Photorhabdus,luminescens,243,phitsanulok,6.8,27,1,Sandy_Loam,LB
19819,434,Photorhabdus,luminescens,255,phitsanulok,6.8,27,1,Sandy_Loam,LB


Final cleanup

In [7]:
metadata.rename(columns = {'ph':'pH'}, inplace=True)
metadata.soiltype = metadata.soiltype.apply(lambda x: x.replace('_', ' ').lower())
metadata[['pH', 'temperature', 'moisture', 'elevation']] = metadata[['pH', 'temperature', 'moisture', 'elevation']].apply(pd.to_numeric)

metadata.head()

,strain,genus,species,elevation,province,pH,temperature,moisture,soiltype,media
19777,409,Xenorhabdus,stockiae,186,phitsanulok,6.8,28,1.0,loam,LB
19789,409,Xenorhabdus,stockiae,186,phitsanulok,6.8,28,1.0,loam,SF900
19817,432,Photorhabdus,luminescens,243,phitsanulok,6.6,26,1.5,sandy loam,LB
19818,433,Photorhabdus,luminescens,243,phitsanulok,6.8,27,1.0,sandy loam,LB
19819,434,Photorhabdus,luminescens,255,phitsanulok,6.8,27,1.0,sandy loam,LB


We save the clean metadata

In [8]:
metadata.index = metadata.index.astype(int)
metadata.to_csv('../data/metadata_clean.csv')

### Merging

In [3]:
# we reload the metabolite data
data_int = pd.read_csv('../data/data_int.csv', index_col=0)
data_auc = pd.read_csv('../data/data_auc.csv', index_col=0)
data_zauc = pd.read_csv('../data/data_zauc.csv', index_col=0)

In [64]:
tag_int = data_int.join(metadata, how='inner')
tag_auc = data_auc.join(metadata, how='inner')
tag_zauc = data_zauc.join(metadata, how='inner')

In [65]:
tag_int.head()

,0,1,2,3,4,5,6,7,8,9,...,strain,genus,species,elevation,province,ph,temperature,moisture,soiltype,media
19777,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,409,Xenorhabdus,stockiae,186,phitsanulok,6.8,28,1,Loam,LB
19789,0.0,0.0,0.0,0.0,86.0,0.0,0.0,0.0,63.0,3.0,...,409,Xenorhabdus,stockiae,186,phitsanulok,6.8,28,1,Loam,SF900
19817,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,24.0,...,432,Photorhabdus,luminescens,243,phitsanulok,6.6,26,1.5,Sandy_Loam,LB
19818,0.0,0.0,0.0,97.0,0.0,0.0,0.0,0.0,0.0,0.0,...,433,Photorhabdus,luminescens,243,phitsanulok,6.8,27,1,Sandy_Loam,LB
19819,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,421.0,36.0,...,434,Photorhabdus,luminescens,255,phitsanulok,6.8,27,1,Sandy_Loam,LB


This yields clean labelled datasets. We will not save these because, in practice, we will very likely do some preprocessing before model construction, and it's easier to do so for the metabolites and metadata separately.